# Pré-processamento e Limpeza (Data Prep)

**Objetivo:** Consolidar as bases de 2022, 2023 e 2024, tratar inconsistências de nomes de colunas (vírgulas, espaços), duplicatas e preparar o dataset final para a Feature Store.

Principais etapas:
- Carga dos dados brutos.
- Padronização de nomes de colunas (mapping).
- Tratamento de duplicatas semânticas (sufixos .1, .2).
- Feature Engineering inicial (pedra numérica, defasagem).

In [1]:
import pandas as pd
import os
import sys

# Config básica para não truncar colunas
pd.set_option('display.max_columns', None)

# Caminho dos arquivos
base_path = '../arquivos'
file_name = 'BASE DE DADOS PEDE 2024 - DATATHON.xlsx'
file_path = os.path.join(base_path, file_name)

if not os.path.exists(file_path):
    print(f"Erro: Arquivo não encontrado em {file_path}")
    # Definindo um caminho absoluto fallback caso o relativo falhe
    file_path = r"C:\Users\Angélica\Desktop\datathon\projeto_datathon\arquivos\BASE DE DADOS PEDE 2024 - DATATHON.xlsx"

# Função utilitária para carga limpa
def load_clean_sheet(file, sheet):
    print(f"Carregando {sheet}...")
    df = pd.read_excel(file, sheet_name=sheet, engine="openpyxl")
    # Remover espaços em branco nos nomes das colunas
    df.columns = df.columns.astype(str).str.strip()
    return df

# Carga das 3 safras
df_2022 = load_clean_sheet(file_path, "PEDE2022")
df_2023 = load_clean_sheet(file_path, "PEDE2023")
df_2024 = load_clean_sheet(file_path, "PEDE2024")

print(f"Shapes carregados: 2022={df_2022.shape}, 2023={df_2023.shape}, 2024={df_2024.shape}")

Carregando PEDE2022...
Carregando PEDE2023...
Carregando PEDE2024...
Shapes carregados: 2022=(860, 42), 2023=(1014, 48), 2024=(1156, 50)


In [2]:
# Checagem de intersecção/diferença de Features (Drift de Schema)
cols_22 = set(df_2022.columns)
cols_23 = set(df_2023.columns)
cols_24 = set(df_2024.columns)

print(f"Colunas [2022 -> 2023]: {len(cols_23 - cols_22)} novas, {len(cols_22 - cols_23)} removidas.")
print(f"Colunas [2023 -> 2024]: {len(cols_24 - cols_23)} novas, {len(cols_23 - cols_24)} removidas.")

# Listar novas colunas em 2024 (potencialmente novas features)
list(cols_24 - cols_23)[:10]

Colunas [2022 -> 2023]: 14 novas, 8 removidas.
Colunas [2023 -> 2024]: 7 novas, 5 removidas.


['Ativo/ Inativo.1',
 'Avaliador6',
 'Escola',
 'INDE 2024',
 'Avaliador5',
 'Pedra 2024',
 'Ativo/ Inativo']

In [3]:
# Função para limpar colunas duplicadas que o pandas renomeia (ex: Coluna.1)
def clean_duplicated_columns(df):
    """
    Remove colunas onde o sufixo '.1', '.2' foi adicionado pelo Pandas
    caso o conteúdo seja idêntico à original.
    """
    # Identificar candidatas a duplicadas
    dup_candidates = [c for c in df.columns if '.' in c and c.split('.')[-1].isdigit()]
    
    to_drop = []
    for dup in dup_candidates:
        original = dup.split('.')[0]
        if original in df.columns:
            # check content equality (handling NaNs)
            if df[original].equals(df[dup]):
                to_drop.append(dup)
                
    if to_drop:
        print(f"Limpando colunas duplicadas exatas: {len(to_drop)}")
        return df.drop(columns=to_drop)
    return df

# Aplicando a limpeza
df_2022 = clean_duplicated_columns(df_2022)
df_2023 = clean_duplicated_columns(df_2023)
df_2024 = clean_duplicated_columns(df_2024)

Limpando colunas duplicadas exatas: 1
Limpando colunas duplicadas exatas: 1


In [4]:
# Verificar consistência de Nomes
from collections import Counter

# Contagem de colunas para detectar renomeações estranhas
# Ex: Algumas vezes 'NOME' vs 'NOME ' aparece, mesmo com .strip() aplicado
# Vamos olhar para as colunas de todos os anos
all_cols = list(df_2022.columns) + list(df_2023.columns) + list(df_2024.columns)
counts = Counter(all_cols)

# Colunas inconsistentes (aparecem em só 1 ou 2 anos)
# Indicativo de que o mapeamento precisa ser ajustado
single_year_cols = [col for col, c in counts.items() if c < 3]
print(f"Colunas presentes em menos de 3 anos: {len(single_year_cols)}")
single_year_cols[:5]

Colunas presentes em menos de 3 anos: 29


['Nome', 'Ano nasc', 'Idade 22', 'Rec Av3', 'Rec Av4']

In [5]:
# Análise rápida de qualidade (Nulos) em 2024
na_counts = df_2024.isnull().sum()
na_pct = (na_counts / len(df_2024)) * 100

# Top 10 colunas com mais nulos
na_stats = pd.DataFrame({'Missing': na_counts, 'Pct': na_pct})
na_stats[na_stats['Missing'] > 0].sort_values('Pct', ascending=False).head(10)

# Padronização de Colunas Texto (Uppercase)
# Importante para joins futuros (ex: Pedra ÁGATA vs Ágata)
categ_cols = df_2024.select_dtypes(include=['object']).columns
for col in categ_cols:
    df_2024[col] = df_2024[col].astype(str).str.upper().str.strip()
    df_2024[col] = df_2024[col].replace('NAN', pd.NA)  # Correção pós-conversão string

print(f"Normalização de strings aplicada em {len(categ_cols)} colunas.")

Normalização de strings aplicada em 21 colunas.


### Definição do Mapping de Variáveis
Para garantir que as safras possam ser concatenadas, precisamos de um dicionário canônico.
As variáveis serão renomeadas para um padrão único (ex: `INDE_24` -> `INDE`).

In [6]:
# --- DICTIONARY MAPPING ---
# Mapeamento manual baseado na análise visual das planilhas
# Estrutura: "NOME_PADRAO": ["VARIAÇÃO_1", "VARIAÇÃO_2", ...]

col_mapping = {
    # Identificadores e Demografia
    "RA": ["RA"],
    "NOME": ["NOME", "NOME ALUNO", "NOME DO ALUNO"],
    "FASE": ["FASE", "FASE TURMA"],
    "TURMA": ["TURMA"],
    "PEDRA": ["PEDRA", "PEDRA 2022", "PEDRA 2023", "PEDRA 2024", "CLASSIFICAÇÃO"],
    "INDE": ["INDE", "INDE 2022", "INDE 2023", "INDE 2024"],
    
    # Indicadores Parciais
    "IAA": ["IAA", "IAA 2022", "IAA 2023", "IAA 2024"],
    "IEG": ["IEG", "IEG 2022", "IEG 2023", "IEG 2024"],
    "IPS": ["IPS", "IPS 2022", "IPS 2023", "IPS 2024"],
    "IDA": ["IDA", "IDA 2022", "IDA 2023", "IDA 2024"],
    "IPP": ["IPP", "IPP 2022", "IPP 2023", "IPP 2024"],
    "IPV": ["IPV", "IPV 2022", "IPV 2023", "IPV 2024"],
    "IAN": ["IAN", "IAN 2022", "IAN 2023", "IAN 2024"],
    
    # Target
    "PONTO_VIRADA": ["PONTO DE VIRADA", "ATINGIU PONTO DE VIRADA", "ATINGIU PV"],
    "DEFASAGEM": ["DEFASAGEM"]
}

def apply_mapping(df, mapping):
    rename_dict = {}
    for standard_name, variations in mapping.items():
        for var in variations:
            # Busca case-insensitive
            current_cols_upper = [c.upper() for c in df.columns]
            var_upper = var.upper()
            
            if var_upper in current_cols_upper:
                # Encontrar o nome real original que bate com a variação
                idx = current_cols_upper.index(var_upper)
                real_col_name = df.columns[idx]
                rename_dict[real_col_name] = standard_name
                
    return df.rename(columns=rename_dict)

# Aplicando padronização
df_2022_std = apply_mapping(df_2022.copy(), col_mapping)
df_2023_std = apply_mapping(df_2023.copy(), col_mapping)
df_2024_std = apply_mapping(df_2024.copy(), col_mapping)

# Check
print("Colunas 'INDE' encontradas:")
print(f"22: {'INDE' in df_2022_std.columns}")
print(f"23: {'INDE' in df_2023_std.columns}")
print(f"24: {'INDE' in df_2024_std.columns}")

Colunas 'INDE' encontradas:
22: False
23: True
24: True


In [7]:
# --- Feature Engineering Inicial ---

# Concatenar para tratamento unificado
df_2022_std['ANO'] = 2022
df_2023_std['ANO'] = 2023
df_2024_std['ANO'] = 2024

# Selecionar apenas colunas padronizadas para o dataset final
cols_interesse = set(col_mapping.keys()) | {'ANO'}
final_cols_22 = [c for c in df_2022_std.columns if c in cols_interesse]
final_cols_23 = [c for c in df_2023_std.columns if c in cols_interesse]
final_cols_24 = [c for c in df_2024_std.columns if c in cols_interesse]

df_full = pd.concat([
    df_2022_std[final_cols_22], 
    df_2023_std[final_cols_23], 
    df_2024_std[final_cols_24]
], ignore_index=True)

# 1. Pipeline de Pedra (Transformar Texto -> Ordinal)
pedra_order = {
    'QUARTZO': 1, 
    'ÁGATA': 2, 'AGATA': 2, 
    'AMETISTA': 3, 
    'TOPÁZIO': 4, 'TOPAZIO': 4
}
if 'PEDRA' in df_full.columns:
    df_full['PEDRA_NUM'] = df_full['PEDRA'].str.upper().map(pedra_order)
    # Quem não tem pedra mapeada fica como 0 ou NaN? Vou assumir NaN por enquanto
    print("Feature PEDRA_NUM gerada.")

# 2. Pipeline de Numéricos (Limpeza de vírgula decimal pt-br)
num_cols = ['INDE', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']
for c in num_cols:
    if c in df_full.columns:
        # Remover caracteres não numéricos exceto ponto e vírgula
        df_full[c] = df_full[c].astype(str).str.replace(',', '.', regex=False)
        df_full[c] = pd.to_numeric(df_full[c], errors='coerce')

print("\nDataset consolidado pronto para análise/modelo.")
print(df_full.info())
df_full.head()

Feature PEDRA_NUM gerada.

Dataset consolidado pronto para análise/modelo.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3030 entries, 0 to 3029
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   RA            3030 non-null   object 
 1   FASE          3030 non-null   object 
 2   TURMA         3030 non-null   object 
 3   NOME          860 non-null    object 
 4   IAA           2865 non-null   float64
 5   IEG           2954 non-null   float64
 6   IPS           2859 non-null   float64
 7   IDA           2852 non-null   float64
 8   PONTO_VIRADA  860 non-null    object 
 9   IPV           2852 non-null   float64
 10  IAN           3030 non-null   float64
 11  ANO           3030 non-null   int64  
 12  INDE          1985 non-null   float64
 13  PEDRA         2023 non-null   object 
 14  IPP           1992 non-null   float64
 15  DEFASAGEM     2170 non-null   float64
 16  PEDRA_NUM     1985 non-null   float64
d

,RA,FASE,TURMA,NOME,IAA,IEG,IPS,IDA,PONTO_VIRADA,IPV,IAN,ANO,INDE,PEDRA,IPP,DEFASAGEM,PEDRA_NUM
0,RA-1,7,A,Aluno-1,8.3,4.1,5.6,4.0,Não,7.278,5.0,2022,NaN,NaN,NaN,NaN,NaN
1,RA-2,7,A,Aluno-2,8.8,5.2,6.3,6.8,Não,6.778,10.0,2022,NaN,NaN,NaN,NaN,NaN
2,RA-3,7,A,Aluno-3,0.0,7.9,5.6,5.6,Não,7.556,10.0,2022,NaN,NaN,NaN,NaN,NaN
3,RA-4,7,A,Aluno-4,8.8,4.5,5.6,5.0,Não,5.278,10.0,2022,NaN,NaN,NaN,NaN,NaN
4,RA-5,7,A,Aluno-5,7.9,8.6,5.6,5.2,Não,7.389,10.0,2022,NaN,NaN,NaN,NaN,NaN


# Próximos Passos (Backlog)
- Salvar `df_full` em parquet ou csv na pasta `src/feature_store` para consumo do modelo.
- Criar feature de `Hitorico_INDE` (lag do ano anterior) se formos usar modelo temporal.
- Validar se a coluna `PONTO_VIRADA` está consistente entre os anos (critério mudou?).